In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="my-test-project", entity="shmiggit")

wandb: Currently logged in as: shmiggit (use `wandb login --relogin` to force relogin)


In [20]:
import pandas as pd
import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa

In [6]:
from projectwind.LSTM_weather_forecast import WindowGenerator, get_LSTM_data
from projectwind.trainer import plot_loss

### Get Data

In [7]:
train_df, val_df, test_df = get_LSTM_data()

### Fetching 25xWTG data ###
### Fetching weather API data ###
### Preparing datasets ###


### Select window and prepare sequences

In [8]:
n_steps_in =  48   # hrs 
n_steps_out = 12   # hrs

window = WindowGenerator(input_width=n_steps_in, label_width=n_steps_out, shift=n_steps_out,
                         train_df=train_df, val_df=val_df, test_df=test_df,
                         input_columns=['Wind Speed', 'Wind_X', 'Wind_Y'],
                         forecast_columns=['windSpeed_API','windGust_API', 
                                        'Wind_API_X', 'Wind_API_Y', 
                                        'WindGust_API_X','WindGust_API_Y'],
                         label_columns=['Wind Speed'])

In [9]:
window

Total window size: 60
Input column name(s): ['Wind Speed', 'Wind_X', 'Wind_Y']
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Forecast column name(s): ['windSpeed_API', 'windGust_API', 'Wind_API_X', 'Wind_API_Y', 'WindGust_API_X', 'WindGust_API_Y']
Forecast indices: [48 49 50 51 52 53 54 55 56 57 58 59]
Label column name(s): ['Wind Speed']
Label indices: [48 49 50 51 52 53 54 55 56 57 58 59]

In [10]:
X_train, X_fc_train, y_train =  window.train
X_val, X_fc_val, y_val  =  window.val
X_test, X_fc_test, y_test  =  window.test

2022-02-18 09:33:06.752133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 09:33:06.761010: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 09:33:06.761363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 09:33:06.762000: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
X_train.shape, X_fc_train.shape, y_train.shape

((6050, 48, 3), (6050, 12, 6), (6050, 12, 1))

In [12]:
X_val.shape, X_fc_val.shape, y_val.shape

((1725, 48, 3), (1725, 12, 6), (1725, 12, 1))

### Build Model

In [13]:
val_performance = pd.DataFrame()
test_performance = pd.DataFrame()

The models below combine two models (1) LSTM model for the performance of the wind turbine & (2) Dense model for the weather forecast. 

These two model outputs are concatenated & ran through two NN Dense layers to produce the output

In [14]:
# Passes LSTM output into combined model
def hybrid_num_model():
    
    # Performance model
    input_perf = tf.keras.layers.Input(shape=(n_steps_in, X_train.shape[2]))
    x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(input_perf)
    x_seq = tf.keras.layers.Dropout(0.2)(x_seq)
    x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(x_seq)
    x_seq = tf.keras.layers.Dropout(0.2)(x_seq)
    x_perf = tf.keras.layers.LSTM(64, return_sequences=False)(x_seq)
    
    # Weather forecast model
    input_fc = tf.keras.layers.Input(shape=(n_steps_out, X_fc_train.shape[2]))
    x = tf.keras.layers.Reshape((n_steps_out*X_fc_train.shape[2],))(input_fc)
    x = tf.keras.layers.Dense(144, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(72, activation='relu')(x)
    x_fc = tf.keras.layers.Dense(n_steps_out, activation="relu")(x)
    
    # Combined model
    combined = tf.keras.layers.concatenate([x_perf, x_fc])
    x = tf.keras.layers.Dense(64, activation="relu")(combined)
    outputs = tf.keras.layers.Dense(n_steps_out, activation="linear")(x)
    
    model = tf.keras.models.Model(inputs=[input_perf, input_fc], outputs=outputs)
    
    return model

In [15]:
def LSTM_model():
    
    input_x = tf.keras.layers.Input(shape=(n_steps_in, X_train.shape[2]))
    x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(input_x)
    #x_seq = tf.keras.layers.Dropout(0.2)(x_seq)
    x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(x_seq)
    #x_seq = tf.keras.layers.Dropout(0.2)(x_seq)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x_seq)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(n_steps_out, activation="linear")(x)
    
    model = tf.keras.models.Model(inputs=input_x, outputs=outputs)
    
    return model

In [16]:
model_name = 'Weather_hybrid_model_1'

In [17]:
model=hybrid_num_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 3)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12, 6)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 48, 64)       17408       input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 72)           0           input_2[0][0]                    
______________________________________________________________________________________________

In [25]:
def compile_and_fit(model, window):
    
    # Early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=30,
                                                      mode='min',
                                                      restore_best_weights=True)

    # Reduce learning rate by an order of magnitude if val_loss does not improve for 20 epoch
    rlrop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.1,
                                                 min_lr=1e-7,
                                                 verbose=1,
                                                 patience=15)
    
    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(learning_rate=0.1),
                  metrics=[tf.metrics.MeanAbsoluteError()])
    
    checkpoint=tf.keras.callbacks.ModelCheckpoint(f"./checkpoint/checkpoint_{model_name}.h5", save_best_only=True, save_weights_only=True)
    
    history = model.fit(x=[X_train,X_fc_train], y=y_train, 
                        epochs=100,
                        validation_data=([X_val, X_fc_val], y_val),
                        callbacks=[early_stopping, checkpoint, rlrop])
    model.save(f'./checkpoint/{model_name}.h5')
    return history

### Run Model

In [26]:
#model = LSTM_model()
history = compile_and_fit(model, window)

Epoch 1/100


2022-02-18 09:42:51.679821: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-02-18 09:42:51.679883: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at matmul_op_impl.h:442 : Internal: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node model/dense/MatMul (defined at opt/conda/lib/python3.7/site-packages/wandb/integration/keras/keras.py:168) ]] [Op:__inference_train_function_31340]

Function call stack:
train_function


### Evaluate Model

In [ ]:
val_performance[model_name] = model.evaluate([X_val, X_fc_val], y_val)
test_performance[model_name] = model.evaluate([X_test, X_fc_test], y_test)

In [ ]:
plot_loss(history)

In [ ]:
y_pred = model.predict([X_val, X_fc_val], batch_size=1)

In [ ]:
plot(X_val, y_pred, y_val)

### Helper - Plot function

In [ ]:
def plot(y_hist, y_pred, y_true, max_subplots=3):
        plot_col = 'Wind Speed'
        plt.figure(figsize=(12, 8))
        plot_col_index = window.column_indices[plot_col]
        max_n = min(max_subplots, len(y_true))
        for n in range(max_n):
            i = np.random.randint(0,len(y_true))
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(window.input_indices, y_hist[i, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if window.label_columns:
                label_col_index = window.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                label_col_index = 0

            plt.plot(window.label_indices, y_true[i, :, label_col_index],
                        label='Labels', c='#2ca02c', marker='.')


            plt.plot(window.label_indices, y_pred[i, :],
                        marker='X', label='Predictions', c='#ff7f0e')
            
            if n == 0:
                plt.legend()

            plt.xlabel('Time [h]')
            plt.tight_layout()

# Load Latest Model

In [ ]:
hn_model = tf.keras.models.load_model('./checkpoint/LSTM_Forecast_Hybrid_num_4Lx16N+18d_fc_2Lx54-18N_3x02do.h5')